In [0]:
import pyspark.sql.functions as F
from pyspark.sql.types import ArrayType, StringType, StructType, StructField
import re

In [0]:
df = spark.read.option("multiline", True).json('dbfs:/user/dblpv13/dblpv13.0.json.gz')

In [0]:
# finds the country names in a list of strings
# modified to only use the first element of the list
# uses regex to remove punctuation from the string and to match the given names of the countries
# can also match the name/abbreviation of the state in the US, in this case it will return the country name 'United States'
# can match the abbreviations 'UK' and 'USA'

def getCountry(s):
    if s is None:
        return None
    arr = []
    countries = ["Afghanistan", "Albania", "Algeria", "Andorra", "Angola", "Antigua & Deps", "Argentina", "Armenia", "Australia", "Austria", "Azerbaijan", "Bahamas", "Bahrain", "Bangladesh", "Barbados", "Belarus", "Belgium", "Belize", "Benin", "Bhutan", "Bolivia", "Bosnia", "Botswana", "Brazil", "Brunei", "Bulgaria", "Burkina", "Burundi", "Cambodia", "Cameroon", "Canada", "Cape Verde", "Central African Republic", "Chad", "Chile", "China", "Colombia", "Comoros", "Congo", "Congo Democratic Republic", "Costa Rica", "Croatia", "Cuba", "Cyprus", "Czech Republic", "Denmark", "Djibouti", "Dominica", "Dominican Republic", "East Timor", "Ecuador", "Egypt", "El Salvador", "Equatorial Guinea", "Eritrea", "Estonia", "Ethiopia", "Fiji", "Finland", "France", "Gabon", "Gambia", "Georgia", "Germany", "Ghana", "Greece", "Grenada", "Guatemala", "Guinea", "Guinea-bissau", "Guyana", "Haiti", "Honduras", "Hungary", "Iceland", "India", "Indonesia", "Iran", "Iraq", "Ireland", "Israel", "Italy", "Ivory Coast", "Jamaica", "Japan", "Jordan", "Kazakhstan", "Kenya", "Kiribati", "South Korea", "Kosovo", "Kuwait", "Kyrgyzstan", "Laos", "Latvia", "Lebanon", "Lesotho", "Liberia", "Libya", "Liechtenstein", "Lithuania", "Luxembourg", "Macedonia", "Madagascar", "Malawi", "Malaysia", "Maldives", "Mali", "Malta", "Marshall Islands", "Mauritania", "Mauritius", "Mexico", "Micronesia", "Moldova", "Monaco", "Mongolia", "Montenegro", "Morocco", "Mozambique", "Myanmar", "Burma", "Namibia", "Nauru", "Nepal", "Netherlands", "New Zealand", "Nicaragua", "Niger", "Nigeria", "Norway", "Romania", "Pakistan", "Palau", "Panama", "Papua New Guinea", "Paraguay", "Peru", "Philippines", "Poland", "Portugal", "Qatar", "Oman", "Russia", "Rwanda", "St Kitts & Nevis", "St Lucia", "Saint Vincent & The Grenadines", "Samoa", "San Marino", "Sao Tome & Principe", "Saudi Arabia", "Senegal", "Serbia", "Seychelles", "Sierra Leone", "Singapore", "Slovakia", "Slovenia", "Solomon Islands", "Somalia", "South Africa", "South Sudan", "Spain", "Sri Lanka", "Sudan", "Suriname", "Swaziland", "Sweden", "Switzerland", "Syria", "Taiwan", "Tajikistan", "Tanzania", "Thailand", "Togo", "Tonga", "Trinidad & Tobago", "Tunisia", "Turkey", "Turkmenistan", "Tuvalu", "Uganda", "Ukraine", "United Arab Emirates", "United Kingdom", "United States", "Uruguay", "Uzbekistan", "Vanuatu", "Vatican City", "Venezuela", "Vietnam", "Yemen", "Zambia", "Zimbabwe"]
    state_names = ["alaska", "alabama", "arkansas", "american samoa", "arizona", "california", "colorado", "connecticut", "district ", "of columbia", "delaware", "florida", "georgia", "guam", "hawaii", "iowa", "idaho", "illinois", "indiana", "kansas", "kentucky", "louisiana", "massachusetts", "maryland", "maine", "michigan", "minnesota", "missouri", "mississippi", "montana", "north carolina", "north dakota", "nebraska", "new hampshire", "new jersey", "new mexico", "nevada", "new york", "ohio", "oklahoma", "oregon", "pennsylvania", "puerto rico", "rhode island", "south carolina", "south dakota", "tennessee", "texas", "utah", "virginia", "virgin islands", "vermont", "washington", "wisconsin", "west virginia", "wyoming"]
    states = ['AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA', 'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME', 'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM', 'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VA', 'VT', 'WA', 'WI', 'WV', 'WY']
    
    for i in s:
        if i["org"] is None:
            arr.append(None)
            break
        sent = re.sub("[^a-zA-Z -]", "", i["org"])
        x = None
        for j in countries:
            x = re.search(j.lower(), sent.lower())
            if x is not None:
                if j.lower() == 'india':
                    x = re.search('indiana', sent.lower())
                    if x is not None:
                        arr.append("United States")
                elif j.lower() == 'georgia':
                    x = re.search('USA', sent)
                    if x is not None:
                        arr.append("United States")
                else:
                    arr.append(j)
                break
        if x is None:
            x = re.search("USA", sent)
            if x is not None:
                arr.append("United States")
                break
        if x is None:
            x = re.search("UK", sent)
            if x is not None:
                arr.append("United Kingdom")
                break
        if x is None:
            x = re.search("england", sent.lower())
            if x is not None:
                arr.append("United Kingdom")
                break
        if x is None:
            x = re.search("scotland", sent.lower())
            if x is not None:
                arr.append("United Kingdom")
                break
        if x is None:
            x = re.search("wales", sent.lower())
            if x is not None:
                arr.append("United Kingdom")
                break
        if x is None:
            for j in states:
                x = re.search(j, sent)
                if x is not None:
                    arr.append("United States")
                    break
        if x is None:
            for j in state_names:
                x = re.search(j, sent.lower())
                if x is not None:
                    arr.append("United States")
                    break
        break
                    
    if len(arr) > 0:
        return arr[0]
    else:
        return None

getCountryUDF = udf(getCountry)

In [0]:
# Organization (affiliation of the first author)
# ID - authors.orgid
# Name - authors.org
# Country - getCountryUDF(F.arrays_zip("authors.org"))
def organization(df):
    new_df = df.select(F.col("authors.orgid").getItem(0).alias("org_id"),
                       F.col("authors.org").getItem(0).alias("org_name"),
                       (getCountryUDF(F.arrays_zip("authors.org"))).alias("org_country"))
    new_df = new_df.na.drop("all")
    return new_df

org_df = organization(df)

In [0]:
# create the new df with new IDs
new_df = org_df.select('org_id', 'org_name', 'org_country').distinct()
new_df = new_df.withColumn('ID', F.monotonically_increasing_id())
new_df = new_df.withColumn('ID', new_df.ID.cast(StringType()))
new_df = new_df.fillna('missing_org_id', 'org_id')

display(new_df)

org_id,org_name,org_country,ID
missing_org_id,Maynard Evans High School,null,0
5f71b3201c455f439fe40a0b,"LIAFA, Université Paris VII, France",France,1
5f71b2dd1c455f439fe3ecd0,"Research Institute for Symbolic Computation, Johannes Kepler University, A-4040 Linz, Austria",Austria,2
5f71b4961c455f439fe4b0fe,"2 Moore Road, Southboro, Massachusetts",United States,3
5f71b2811c455f439fe3c5b7,"Satellite Television Corp., New York, NY|c|",United States,4
5f71b3311c455f439fe41212,"Yandex, Moscow, Russian Fed.",Russia,5
missing_org_id,Mainz,null,6
5f71b2a01c455f439fe3d2b7,"University of Illinois at Urbana-Champaign Department of Philosophy 105 Gregory Hall, MC 468 Urbana IL 61801 USA",United States,7
5f71b2f51c455f439fe3f795,Sony CSL Interaction Lab,null,8
5f71b2ca1c455f439fe3e4a2,"Univ. of Nijmegen, The Netherlands",Netherlands,9


In [0]:
# add separate org id and name columns to the original df
df2 = df.withColumn("org_id", F.col("authors.orgid").getItem(0)).withColumn("org_name", F.col("authors.org").getItem(0))
df2 = df2.fillna('missing_org_id', 'org_id')

In [0]:
# join newly created IDs to the original df and replace missing org ids with generated IDs
final_df = df2.join(new_df.select('ID', 'org_id', 'org_name'), on=['org_id', 'org_name'])
final_df = final_df.replace('missing_org_id', None)
final_df = final_df.withColumn('org_id', F.coalesce(final_df.org_id, final_df.ID))
final_df = final_df.drop('ID', 'org_name')

display(final_df)

org_id _id abstract authors doi fos isbn issn issue keywords lang n_citation page_end page_start pdf references title url venue volume year 5f71b2aa1c455f439fe3d5f8 53e99785b7602d9701f40583 Summary: We implemented a software tool called GENESIS for three different genome rearrangement problems: Sorting a unichromosomal genome by weighted reversals and transpositions (SwRT), sorting a multichromosomal genome by reversals, translocations, fusions and fissions (SRTl), and sorting a multichromosomal genome by weighted reversals, translocations, fusions, fissions and transpositions (SwRTTl). Availability: Source code can be obtained by the authors, or use the web interface http://www.uni-ulm.de/in/theo/research/genesis.html Contact: simon.gog@uni-ulm.de List(List(53f43018dabfaeb22f42a3cf, null, null, null, Simon Gog, null, null, null, null, Univ Ulm, Inst Theoret Comp Sci, D-89069 Ulm, Germany, null, 5f71b2aa1c455f439fe3d5f8, List(Univ Ulm, Inst Theoret Comp Sci, D-89069 Ulm, Germany), null, null), List(53f42cfadabfaedf43510fe2, null, null, null, Martin Bader, null, null, null, null, Univ Ulm, Inst Theoret Comp Sci, D-89069 Ulm, Germany, null, 5f71b2aa1c455f439fe3d5f8, List(Univ Ulm, Inst Theoret Comp Sci, D-89069 Ulm, Germany), null, null), List(53f42c6edabfaedd74d2898b, null, null, null, Enno Ohlebusch, null, null, null, null, Univ Ulm, Inst Theoret Comp Sci, D-89069 Ulm, Germany, null, 5f71b2aa1c455f439fe3d5f8, List(Univ Ulm, Inst Theoret Comp Sci, D-89069 Ulm, Germany), null, null)) 10.1093/bioinformatics/btn026 null Handbook of Hope 5 List(software tool, unichromosomal genome, web interface, source code, different genome rearrangement problem, multichromosomal genome, html Contact, weighted reversal) en null 712 711 null null GENESIS List(http://doi.ieeecomputersociety.org/10.1109/SERVICES-I.2009.108, http://www.webofknowledge.com/) List(53a7284920f7420be8baedee, null, null, null, null, null, null, SERVICES I, null, null, null, null, null) 24 2008 5f71b2ce1c455f439fe3e62a 53e99785b7602d9701f440df List(List(53f46a0adabfaec09f24e389, null, Petra.Stoerig@uni-duesseldorf.de, 5b869f98e1cd8e14a3b1e5b9, Petra Stoerig, null, null, null, null, Institute of Experimental Psychology II, Heinrich-Heine-University, Duesseldorf 40225, Germany, null, 5f71b2ce1c455f439fe3e62a, null, null, null), List(53f4682bdabfaefedbb8fa5e, null, Alan.Cowey@psy.ox.ac.uk, 5b86b6ede1cd8e14a34deaa0, Alan Cowey, null, null, null, null, Department of Experimental Psychology, University of Oxford, Oxford OX1 3UD, UK, null, 5f71b2841c455f439fe3c6ab, null, null, null)) 10.1016/j.cub.2007.07.016 null 0960-9822 19 List() en 0 4 R822 null null Blindsight. List(http://www.scholarpedia.org/article/Blindsight, http://www.ncbi.nlm.nih.gov/pubmed/17925204?report=xml&format=text) List(555036c97cea80f954155dd5, null, null, null, null, null, null, Scholarpedia, null, null, null, null, 0) 17 2007 5f71b2811c455f439fe3c5cb 53e99792b7602d9701f58126 E-government has emerged not merely as a specialization in public administration, but as a transformative force affecting all levels and functions of government. This volume, written by a mix of practitioners and researchers, provides an overview of the management challenges and issues involved in seeking a new form of governance -- digital government. An initial introductory section presents the hopes for e-government and outlines its history in the United States and globally. Section II, "The New Face of Government," examines FirstGov, the premiere example of e-government and surveys its political implications. Section III, "Issues in Digital Governance," discusses such management challenges as privacy rights, e-procurement, e-commerce, and ethics in e-government. Section IV, "Preparing for Digital Government," discusses data warehousing and related prerequisites for e-government, including the education and training of the public service. Finally, Section V, "The Future of E-Government," discusses the digital divide, citizen particip

In [0]:
# replace missing org ids with generated IDs
new_df = new_df.replace('missing_org_id', None)
new_df = new_df.withColumn('org_id', F.coalesce(new_df.org_id, new_df.ID))
new_df = new_df.drop('ID')
display(new_df)

org_id,org_name,org_country
0,Maynard Evans High School,null
5f71b3201c455f439fe40a0b,"LIAFA, Université Paris VII, France",France
5f71b2dd1c455f439fe3ecd0,"Research Institute for Symbolic Computation, Johannes Kepler University, A-4040 Linz, Austria",Austria
5f71b4961c455f439fe4b0fe,"2 Moore Road, Southboro, Massachusetts",United States
5f71b2811c455f439fe3c5b7,"Satellite Television Corp., New York, NY|c|",United States
5f71b3311c455f439fe41212,"Yandex, Moscow, Russian Fed.",Russia
6,Mainz,null
5f71b2a01c455f439fe3d2b7,"University of Illinois at Urbana-Champaign Department of Philosophy 105 Gregory Hall, MC 468 Urbana IL 61801 USA",United States
5f71b2f51c455f439fe3f795,Sony CSL Interaction Lab,null
5f71b2ca1c455f439fe3e4a2,"Univ. of Nijmegen, The Netherlands",Netherlands


In [0]:
# Most productive organizations. Sort organizations by total no of articles released by that organization. Take top 10.
# Unique IDs where org_id null
display(final_df.groupby('org_id').count().orderBy(F.desc('count')).take(10))

org_id,count
5f71b5c41c455f439fe532ea,1287
5f71b2861c455f439fe3c771,946
5f71b2811c455f439fe3c5cb,931
5f71b28b1c455f439fe3c989,815
5f71b2841c455f439fe3c67b,766
5f71b2831c455f439fe3c634,709
5f71b2841c455f439fe3c6b7,649
5f71b2831c455f439fe3c633,642
5f71b5e01c455f439fe53f9a,604
5f71b2a01c455f439fe3d2b7,570


In [0]:
# save as a delta table
new_df.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable("organizations")

In [0]:
display(spark.table("organizations").limit(10))

org_id,org_name,org_country,ID
null,Maynard Evans High School,null,0
5f71b3201c455f439fe40a0b,"LIAFA, Université Paris VII, France",France,1
5f71b2dd1c455f439fe3ecd0,"Research Institute for Symbolic Computation, Johannes Kepler University, A-4040 Linz, Austria",Austria,2
5f71b4961c455f439fe4b0fe,"2 Moore Road, Southboro, Massachusetts",United States,3
5f71b2811c455f439fe3c5b7,"Satellite Television Corp., New York, NY|c|",United States,4
5f71b3311c455f439fe41212,"Yandex, Moscow, Russian Fed.",Russia,5
null,Mainz,null,6
5f71b2a01c455f439fe3d2b7,"University of Illinois at Urbana-Champaign Department of Philosophy 105 Gregory Hall, MC 468 Urbana IL 61801 USA",United States,7
5f71b2f51c455f439fe3f795,Sony CSL Interaction Lab,null,8
5f71b2ca1c455f439fe3e4a2,"Univ. of Nijmegen, The Netherlands",Netherlands,9
